In [1]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('data/Artificial_proteins.csv')[['ID', 'Sequence', 'Tool']]
df = df[df['Tool'] != 'Real'].reset_index(drop=True)

In [16]:
CHUNKS = 1000

### Split analysis into smaller chunks

In [17]:
seqs = df.apply(lambda row: f'>{str(row["ID"])}\n{row["Sequence"]}', axis=1)

In [18]:
seqs_chunks = np.array_split(seqs, CHUNKS)

### Run BLAST search for chunks

In [ ]:
for i in range(CHUNKS): #TODO
    fasta = '\n'.join(seqs_chunks[i])
    print(f"Analyzing {i}")
    result_handle = NCBIWWW.qblast("blastp", "nr", fasta)
    blast_results = result_handle.read()
    with open(f'Blast/results_{i}.xml', 'w') as save_file: 
        save_file.write(blast_results)

### Process chunk results

In [ ]:
blast_file = open("data/Artificial_proteins_Blast.csv", 'w')
blast_file.write('ID,Match_title,Match_e-value,Match_sequence\n')

In [ ]:
for i in range(CHUNKS):
    print(i)
    for record in NCBIXML.parse(open(f'Blast/results_{i}.xml')): 
        blast_file.write(record.query + ',')
        if record.alignments:
            blast_file.write(f"{record.alignments[0].title},{record.alignments[0].hsps[0].expect},{record.alignments[0].hsps[0].sbjct}\n")
        else:
            blast_file.write(',,\n')

In [27]:
blast_file.close()

-----------------------------------